In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [2]:
# !conda install -y --channel conda-forge pyvips
!conda install ../input/pyvips-install/pyvips/*.tar.bz2


######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [3]:
import cv2
from openslide import OpenSlide
import tifffile as tiff
import os
import gc

In [4]:
import glob

# test_image_paths = glob.glob("../input/mayo-clinic-strip-ai/train/*.tif")
test_image_paths = glob.glob("../input/mayo-clinic-strip-ai/test/*.tif")

In [5]:
# scale = 4
# output_dir = "/kaggle/working/"
# too_big_for_process = []
# # for path in test_image_paths[180:200]:
# for path in test_image_paths:

#     print(path)
#     slide = OpenSlide(path)

#     if slide.dimensions[0]*slide.dimensions[1] < 4131662535:
#         image_id = os.path.splitext(os.path.basename(path))[0]
#         image = tiff.imread(path)
#         print(f"{image.shape}")
#         cv2.imwrite(os.path.join(output_dir, f"{image_id}.jpg"), image[::scale,::scale,::-1])
#         del image
#         gc.collect()
#     else:
#         print("Skip process for avoiding OOM possibility.")
#         too_big_for_process.append(path)

# test_image_paths = glob.glob("/kaggle/working/*.jpg") 

In [6]:
import zipfile
import torch

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
# from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torchvision import models
import torch.nn as nn
import timm
import torchvision
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import pyvips
import scipy.stats
import random
from fastai.vision import *
from fastai.layers import AdaptiveConcatPool2d, Flatten, Mish

In [7]:
class AttentionSoftMax(torch.nn.Module):
    def __init__(self, in_features=3, out_features = None):
        super(AttentionSoftMax, self).__init__()
        self.otherdim = 'b'
        if out_features is None:
            out_features = in_features

        self.layer_linear_tr = nn.Linear(in_features, out_features) 
        self.activation = nn.LeakyReLU() 
        self.layer_linear_query= nn.Linear(out_features, 1)

    def forward(self, x):
        keys = self.layer_linear_tr(x)
        keys = self.activation(keys)

        attention_map_raw = self.layer_linear_query(keys)[...,0]
        attention_map = nn.Softmax(dim=-1)(attention_map_raw)

        result = torch.einsum(f'{self.otherdim}i, {self.otherdim}ij->{self.otherdim}j', attention_map, x)
        return result

class Model (nn.Module):
    def __init__(self, arch='tf_efficientnetv2_s', n=2, pre=False, enc_out_feat=1280): 
        super().__init__()
        m = timm.create_model(model_name=arch, pretrained = pre, num_classes = 0)
        self.enc = nn.Sequential(*list(m.children())[:-1])
        self.enc_out_feat = enc_out_feat
        self.head = nn.Sequential(AttentionSoftMax(enc_out_feat), nn.Dropout(0.5), nn.Linear(enc_out_feat,n))

    def forward(self, x):
#         print(x.shape)
        bs, instance_num, c, w, h = x.shape 
        x = x.view(bs*instance_num,c,w,h)

        # print(x.shape)
        #x: bs*instance_num x cx wxh

        x = self.enc(x)
        #x: bs instance_num x enc_out_feat

        x = x.view(bs,instance_num, self.enc_out_feat).contiguous()

        #x: bs x instance_num x enc_out_feat
        x = self.head(x)
        #x: bs x n

        return x

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

tile_sz=384
sz = 384
max_size = 10000
N=16

cuda:0


In [9]:
# n_class = 2
# backbone = timm.create_model(model_name, pretrained=False, num_classes=0).to(device)
# model = nn.Sequential(
#     backbone,
#     nn.Dropout(0.2),
#     nn.Linear(backbone.num_features, n_class)
# ).to(device)

model = Model().to(device)

# model_path = "../input/mayoinfermodelbfjctp/20220821_models/kaggle/working/models/*.pth"
model_path = "../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/*.pth"
model_paths = glob.glob(model_path)

model_paths

['../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold5.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold3.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold8.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold7.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold4.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold1.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold0.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-effv2-fold6.pth',
 '../input/mayo-infer-model-bfj/models-attention/kaggle/working/models/concat-tile-pooling-384-e

In [10]:
def get_cutting_point(image, width_0, height_0, box_size=20000):
    height, width, c = image.shape
    multiplier_h = height/height_0
    multiplier_w = width/width_0

    box_h = int(box_size*multiplier_h)
    box_w = int(box_size*multiplier_w)

    scores = []
    for i in range(height//int(box_h)+1):
        for j in range(width//int(box_w)+1):
            start_y = i*box_h 
            end_y = (i+1)*box_h
            start_x = j*box_w
            end_x = (j+1)*box_w

            if i == height//box_h:
                start_y = height - box_h
                end_y = height
            if j == width//box_w:
                start_x = width - box_w
                end_x = width

            if start_x < 0:
                start_x = 0
            if start_y < 0:
                start_y = 0

            scores.append((start_x, start_y, len(cv2.imencode(".jpg", image[start_y:end_y, start_x:end_x])[1])))

    x_pos , y_pos, _ = pd.DataFrame(scores, columns=["start_x", "start_y", "score"]).sort_values('score', ascending=False).iloc[0, :]
    
    return int((x_pos/width)*width_0), int((y_pos//height)*height_0)

In [11]:
def tile(img, sz=128, N=16):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    scores = []
    for im in img:
        scores.append(len(cv2.imencode(".jpg", im)[1]))
#     idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
#     img = img[idxs]
    scores, img = zip(*sorted(zip(scores, img), reverse=True, key=lambda x: x[0]))
#     high_info_ind = pd.Series(scores).where(pd.Series(scores) > 30000).idxmin()
#     print(high_info_ind is not np.nan)
    
    bg_ind = pd.Series(scores).where(pd.Series(scores) > 10000).idxmin()
    
    try:
        bg_cand = img[bg_ind:]
        for bgi, bg in enumerate(bg_cand):
            bg = bg.reshape(bg.shape[0] * bg.shape[1], bg.shape[2])
            white, _ = scipy.stats.mode(bg, axis=0)
            diff_to_white = (255,255,255) - white
            if sum(diff_to_white[0]) < 128*3:
                break
            else:
                diff_to_white = [[0,0,0]]
    except TypeError:
        print("cannot find bg")
        diff_to_white = [[0,0,0]]
        bg_cand = [np.zeros((sz,sz,3))]
        bgi = 0

    return img[:N], bg_cand[bgi], diff_to_white

def vips2numpy(vi):
    format_to_dtype = {
       'uchar': np.uint8,
       'char': np.int8,
       'ushort': np.uint16,
       'short': np.int16,
       'uint': np.uint32,
       'int': np.int32,
       'float': np.float32,
       'double': np.float64,
       'complex': np.complex64,
       'dpcomplex': np.complex128,
    }
    return np.ndarray(buffer=vi.write_to_memory(),dtype=format_to_dtype[vi.format],shape=[vi.height, vi.width, vi.bands])

def return_tiled_images(image_path, transform, N=16, max_size=20000, crop_size=384):
    image = pyvips.Image.thumbnail(image_path, max_size)
    image = vips2numpy(image)
    width, height, c = image.shape
    print(f"Input width: {width} height: {height}")
    images, bg, diff_to_white = tile(image, sz=crop_size, N=N)
    output_images = []
    for idx, img in enumerate(images):
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#         img = cv2.imencode(".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 100])[1]
#         img = cv2.imdecode(img, flags=cv2.IMREAD_COLOR)
        img = img + diff_to_white
        img = img / img.max()
        img = np.clip(img * 255, a_min = 0, a_max = 255).astype(np.uint8)
        img = transform(img)
        
        output_images.append(img)
    
    img_indexes = random.sample(list(range(0, N)), N)
    
    batched_images = []
    one_batch = []
    for i, index in enumerate(img_indexes):
        one_batch.append(output_images[index])
        if i%N==(N-1):
            batched_images.append(torch.stack(one_batch, dim=0))
            one_batch = []

    batched_images = torch.stack(batched_images, dim=0)
#     del img, image, images, output_images; gc.collect()
    return batched_images

In [12]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((sz, sz)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

preds = []

for path in test_image_paths:
    thumb = pyvips.Image.thumbnail(path, 500)
    thumb = vips2numpy(thumb)
    slide = OpenSlide(path)
    height, width, c = thumb.shape
    width_0, height_0 = slide.dimensions

    print(f"Input width: {width} height: {height}")
    print(f"org width: {width_0} height: {height_0}")

    x_pos, y_pos = get_cutting_point(thumb, width_0, height_0, max_size)
    print(path, x_pos, y_pos)
    cropped = slide.read_region((x_pos, y_pos), 0, (max_size, max_size))
    images, bg, diff_to_white = tile(img=cv2.cvtColor(np.array(cropped), cv2.COLOR_RGB2BGR), sz=tile_sz, N=N)
    
    batched_image = []
    for image in images:
         batched_image.append(transform(image))
    
    batched_image = torch.stack(batched_image, dim=0).to(device)
    
    del images, image
    gc.collect()
    
    for i, m_path in enumerate(model_paths):
        model.load_state_dict(
        torch.load(
            m_path, map_location=device
            )
        )
        model.train(False)
        with torch.cuda.amp.autocast():
            pred = model(torch.unsqueeze(batched_image, 0))
#             pred = model(images)

        sm = nn.Softmax(dim=1)
        pred = sm(pred).to('cpu').detach().numpy().copy()
        
        pred_ce = (pred[:, 0]).mean()
        pred_laa = (pred[:, 1]).mean()
        
        preds.append((path, i, pred_ce, pred_laa))
        
    del batched_image
    gc.collect()

Input width: 280 height: 500
org width: 34007 height: 60797
../input/mayo-clinic-strip-ai/test/006388_0.tif 19918 0
Input width: 123 height: 500
org width: 15255 height: 61801
../input/mayo-clinic-strip-ai/test/00c058_0.tif 0 0
Input width: 100 height: 500
org width: 5946 height: 29694
../input/mayo-clinic-strip-ai/test/008e5c_0.tif 0 0
Input width: 500 height: 238
org width: 55831 height: 26553
../input/mayo-clinic-strip-ai/test/01adc5_0.tif 0 0


In [13]:
# if len(too_big_for_process)>0:
#     for i in too_big_for_process:
#         preds.append((i, 0, 0.82, 0.28))

In [14]:
def path_to_patient_id(path):
    return os.path.basename(path).split("_")[0]

df = pd.DataFrame(preds, columns=("path", "fold", "CE", "LAA"))
df["patient_id"] = df["path"].map(path_to_patient_id)
df.groupby("patient_id").mean().drop("fold", axis=1).to_csv("submission.csv")

In [15]:
df.groupby("patient_id").mean().drop("fold", axis=1)

,CE,LAA
patient_id,,
006388,0.727539,0.272461
008e5c,0.731445,0.268799
00c058,0.740234,0.259521
01adc5,0.732910,0.266846
